In [1]:
%matplotlib QT5
import numpy as np
import matplotlib.pyplot as plt

import pyaccel
import pymodels
from apsuite import WIP_lattice_errors
from apsuite.orbcorr import OrbitCorr, CorrParams
from mathphys.functions import save_pickle, load_pickle

Creating errors configs

In [2]:
dips_error = WIP_lattice_errors.DipolesErrors()
quads_error = WIP_lattice_errors.QuadsErrors()
quads_skew_error = WIP_lattice_errors.QuadsSkewErrors()
sexts_error = WIP_lattice_errors.SextsErrors()
girder_error = WIP_lattice_errors.GirderErrors()
bpms_error = WIP_lattice_errors.BpmsErrors()
error_configs = [dips_error, quads_error, sexts_error, quads_skew_error,
                 bpms_error, girder_error]

Generate errors and save in a file

In [3]:
# get family data
model = pymodels.si.create_accelerator()
model.cavity_on = False
model.radiation_on = 0
famdata = pymodels.si.families.get_family_data(model)
spos = pyaccel.lattice.find_spos(model, indices='open')
# create a seed
seed = 144641

# create manage errors object
lattice_errors = WIP_lattice_errors.ManageErrors()
lattice_errors.nr_mach = 10
lattice_errors.nominal_model = model
lattice_errors.famdata = famdata
lattice_errors.spos = spos
lattice_errors.seed = seed
lattice_errors.error_configs = error_configs
lattice_errors.cutoff = 1
lattice_errors.do_bba = True

cparams = CorrParams()
cparams.minsingval = 0.2
cparams.tolerance = 1e-9
cparams.maxnriters = 20
lattice_errors.ocorr_params = cparams

In [4]:
errors = lattice_errors.generate_errors(save_errors=True)

Load error file

In [4]:
data = lattice_errors.load_error_file('errors')
print(lattice_errors.nr_mach)
print(data.keys())
print('B1:')
print(data['B1'].keys())
print(np.shape(data['B1']['index']))
print(np.shape(data['B1']['posx']))
print('QFA:')
print(data['QFA'].keys())
print(np.shape(data['QFA']['index']))
print(np.shape(data['QFA']['posx']))
print(data['QFA']['multipoles'].keys())

10
dict_keys(['B1', 'B2', 'BC', 'QFA', 'QDA', 'Q1', 'Q2', 'Q3', 'Q4', 'QDB1', 'QFB', 'QDB2', 'QDP1', 'QFP', 'QDP2', 'SFA0', 'SDA0', 'SDA1', 'SFA1', 'SDA2', 'SDA3', 'SFA2', 'SFB2', 'SDB3', 'SDB2', 'SFB1', 'SDB1', 'SDB0', 'SFB0', 'SFP2', 'SDP3', 'SDP2', 'SFP1', 'SDP1', 'SDP0', 'SFP0', 'QS', 'BPM', 'girder'])
B1:
dict_keys(['posx', 'posy', 'roll', 'pitch', 'yaw', 'excit', 'kdip', 'multipoles', 'index'])
(40, 30)
(10, 40)
QFA:
dict_keys(['posx', 'posy', 'roll', 'pitch', 'yaw', 'excit', 'multipoles', 'index'])
(10, 1)
(10, 10)
dict_keys(['normal', 'skew', 'r0'])


In [5]:
lattice_errors.configure_corrections()

Configuring orbit correction...
Configuring optics correction...
Configuring tune correction...
Nominal tunes: 49.0960 14.1518
Configuring coupling correction...


In [6]:
lattice_errors.nr_mach = 10
# correction without girder errors: 5 steps
# lattice_errors.do_bba = False
# lattice_errors.use_nominal_jacobian = True
# lattice_errors.ocorr_params.minsingval = 0.1
# lattice_errors.ocorr_params.maxnriters = 20
# lattice_errors.ocorr_params.tolerance = 1e-8

# correction with girder errors: 8 steps

lattice_errors.set_sext_to_zero = False
lattice_errors.ramp_multipoles = False
lattice_errors.ramp_tunes = True
lattice_errors.ramp_coup = True
lattice_errors.ramp_optics = True
lattice_errors.apply_girder = True
lattice_errors.rescale_girder = 1

lattice_errors.do_bba = True
lattice_errors.use_nominal_jacobian = True
lattice_errors.ocorr_params.minsingval = 0.2
lattice_errors.ocorr_params.maxnriters = 20
lattice_errors.ocorr_params.tolerance = 1e-6
machines_wo_mp_corr = lattice_errors.generate_machines(nr_steps=8)

Machine  0
Step  1
Applying errors...Done!
Correcting orbit...Done!

Optics correction tolerance achieved:  False
Old tunes: 49.0957 14.1517
iter # 1 - New tunes: 49.0960 14.1518
iter # 2 - New tunes: 49.0960 14.1518
Minimum tune separation before corr: 0.400 %
Minimum tune separation after corr: 0.004 %
Step  2
Applying errors...Done!
Correcting orbit...Done!

Optics correction tolerance achieved:  False
Old tunes: 49.0956 14.1516
iter # 1 - New tunes: 49.0960 14.1518
iter # 2 - New tunes: 49.0960 14.1518
Minimum tune separation before corr: 0.406 %
Minimum tune separation after corr: 0.006 %
Step  3
Applying errors...Done!
Correcting orbit...Done!

Optics correction tolerance achieved:  False
Old tunes: 49.0954 14.1515
iter # 1 - New tunes: 49.0960 14.1518
iter # 2 - New tunes: 49.0960 14.1518
Minimum tune separation before corr: 0.412 %
Minimum tune separation after corr: 0.005 %
Step  4
Applying errors...Done!
Correcting orbit...Done!

Optics correction tolerance achieved:  False
O

In [ ]:
save_pickle(machines_wo_mp_corr, 'machines_w/o_multipole_corr', overwrite=True)
# save_pickle(machines, 'machines_w_multipole_corr', overwrite=True)

In [ ]:
lattice_errors.nr_mach = 10
# correction without girder errors: 5 steps
# lattice_errors.do_bba = False
# lattice_errors.use_nominal_jacobian = True
# lattice_errors.ocorr_params.minsingval = 0.1
# lattice_errors.ocorr_params.maxnriters = 20
# lattice_errors.ocorr_params.tolerance = 1e-8

# correction with girder errors: 8 steps

lattice_errors.set_sext_to_zero = False
lattice_errors.ramp_multipoles = False
lattice_errors.ramp_tunes = True
lattice_errors.ramp_coup = True
lattice_errors.ramp_optics = True
lattice_errors.apply_girder = True
lattice_errors.rescale_girder = 1

lattice_errors.do_bba = True
lattice_errors.use_nominal_jacobian = True
lattice_errors.ocorr_params.minsingval = 0.2
lattice_errors.ocorr_params.maxnriters = 20
lattice_errors.ocorr_params.tolerance = 1e-6
machines_wo_mp_corr = lattice_errors.generate_machines(nr_steps=8)